## Set Up

In [ ]:
import boto3
import pandas as pd
import numpy as np

from scipy import stats

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV


In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Import Tesla Data From S3

In [ ]:
## create an S3 client
s3 = boto3.client('s3')

## specify the S3 object keys
bucket_name = 'tesla-project'
file_keys = ['data/tesla_used_car_sold-2022-05.csv', 
             'data/tesla_used_car_sold-2022-06.csv',
             'data/tesla_used_car_sold-2022-07.csv', 
             'data/tesla_used_car_sold-2022-08.csv']

## read the S3 objects and concatenate into a single dataframe
dfs = []
for file_key in file_keys:
    obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    dfs.append(pd.read_csv(obj['Body']))
df = pd.concat(dfs, ignore_index=True)

## Data Preprocessing & Feature Engineering

#### Vehicle Specs

In [ ]:
## Perform one hot encoding on each of the vehicle features in the 'features' column
df = pd.concat([df, pd.get_dummies(df['features'].str.split(';').apply(pd.Series).stack()).groupby(level=0).sum()], axis=1)

In [ ]:
## Remove 'Model S', 'Model X', 'Model 3', 'Model Y' prefixes from trim values
df['trim'] = df['trim'].str.replace('Model S ', '').str.replace('Model X ', '').str.replace('Model 3 ', '').str.replace('Model Y ', '')

## Perform one hot encoding for the trim
df = pd.concat([df, pd.get_dummies(df['trim']).astype('int')], axis=1)

In [ ]:
## Perform one hot encoding for the vehicle model
df = pd.concat([df, pd.get_dummies(df['model']).astype('int')], axis=1)

#### Location

In [ ]:
## Extracting the city from the location column and saving it in title case
df['location'] = df['location'].str.split(',').str[0].str.title()

## Dropping rows with missing locations
df = df.loc[~df['state'].isna()]

## Fill nulls for metro area
df['metro'].fillna('No Metro', inplace=True)

#### Date

In [ ]:
## Convert date to days since epoch
df['sold_date'] = pd.to_datetime(df['sold_date'])
df['daysSinceEpoch'] = (df['sold_date'] - pd.Timestamp("1970-01-01")).dt.days

#### Location

In [ ]:
# Create a label encoder object
le = LabelEncoder()

# Encode the metro column
df['metro_label'] = le.fit_transform(df['metro'])

In [ ]:
# Encode the location column
df['location_label'] = le.fit_transform(df['location'])

# Concatenate the location and state columns with a hyphen
df['location_state'] = df['location'] + '-' + df['state']

# Encode the location_state column
df['location_state_label'] = le.fit_transform(df['location_state'])

### Add Calculated Fields

In [ ]:
# Calculate the age of the vehicle in years
df['vehicle_age'] = (df['sold_date'].dt.year - df['year']).apply(lambda x: max(x,1))

# Calculate miles per year
df['miles_per_year'] = df['miles'] // df['vehicle_age']

In [ ]:
## Calculate the zScore of the mileage
df['mileage_zscore'] = stats.zscore(df['miles'])

### Import Lookup Table

In [ ]:
## Import lookup table from s3 and save in a dataframe
obj = s3.get_object(Bucket=bucket_name, Key='data/tesla_project_economic_indicator_data.csv')
lookup = pd.read_csv(obj['Body'])
lookup.drop(columns='Year', inplace=True)

### Compile

In [ ]:
df = df.merge(lookup, left_on = df.sold_date.dt.month, right_on = lookup.Month)

## Exploratory Data Analysis

In [ ]:
# group data by model and sold date, and calculate the mean sold price per week
grouped_df = df.groupby(["model", pd.Grouper(key="sold_date", freq="W")])["sold_price"].mean().reset_index()

# pivot data to have models as columns
pivoted_df = grouped_df.pivot(index="sold_date", columns="model", values="sold_price")

# plot time series
pivoted_df.plot(figsize=(10, 6))
plt.title("Average Tesla Model Prices By Week")
plt.xlabel("Sold Date")
plt.ylabel("Sold Price")
plt.show()


In [ ]:
# Group by model and sold_date, resampled to weekly frequency, and count the number of sales
weekly_sales = df.groupby(['model', pd.Grouper(key='sold_date', freq='W')]).count().reset_index()

# Pivot the table to have models as columns and the count of sales as values
weekly_sales_pivot = weekly_sales.pivot(index='sold_date', columns='model', values='sold_price')

# Plot the counts of each Tesla model sold over time
weekly_sales_pivot.plot(kind='line', figsize=(10,5))
plt.title('Counts of Tesla Models Sold By Week')
plt.xlabel('Week')
plt.ylabel('Count')
plt.show()

## Model Prep

#### Final Feature Selection

In [ ]:
columns_to_exclude = ['vin', 'location', 'state', 'metro', 'location_state', 'location_label', 'color', 'interior', 'wheels', 'features', 'country', 'currency', 'trim', 'model', 'sold_date', 'key_0', 'Month', '']


In [ ]:
model_columns = [column for column in df.columns.to_list() if column not in columns_to_exclude]

In [ ]:
model_df = df[model_columns]

In [ ]:
model_df.describe()

### Splitting the Dataset

In [ ]:
X = model_df.drop('sold_price', axis=1)
y = model_df['sold_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Predictive Models

### Random Forest

In [ ]:
# Fit the random forest model
rf_model = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Calculate the mean absolute error on the test set
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print('Mean Absolute Error:', mae_rf)

In [ ]:
important_feats = rf_model.feature_importances_

sorted_idx = np.argsort(important_feats)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.barh(pos, important_feats[sorted_idx], align='center')
plt.yticks(pos, X_test.columns[sorted_idx])
plt.ylim([len(X_train.columns)-25, len(X_train.columns)])
plt.title('Feature Importance')
plt.show()

### XGBoost

In [ ]:
# Define model
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=7)

# Train model
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred_xgb = xgb_model.predict(X_test)

# Calculate mean absolute error
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print("Mean Absolute Error:", mae_xgb)

In [ ]:
important_feats = xgb_model.feature_importances_

sorted_idx = np.argsort(important_feats)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.barh(pos, important_feats[sorted_idx], align='center')
plt.yticks(pos, X_test.columns[sorted_idx])
plt.ylim([len(X_train.columns)-25, len(X_train.columns)])
plt.title('Feature Importance')
plt.show()

### Gradient Boost

In [ ]:
# create the model
gbr = GradientBoostingRegressor(loss='absolute_error', learning_rate=0.1, n_estimators=100, max_depth=7, random_state=42)

# fit the model to the training data
gbr_model = gbr.fit(X_train, y_train)

# predict on the test set
y_pred_gbr = gbr_model.predict(X_test)

# calculate mean absolute error
mae_gbr = mean_absolute_error(y_test, y_pred_gbr)
print("Mean Absolute Error:", mae_gbr)

In [ ]:
important_feats = gbr_model.feature_importances_

sorted_idx = np.argsort(important_feats)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.barh(pos, important_feats[sorted_idx], align='center')
plt.yticks(pos, X_test.columns[sorted_idx])
plt.ylim([len(X_train.columns)-25, len(X_train.columns)])
plt.title('Feature Importance')
plt.show()